# Toric Decoder
Apply the [2D decoder](https://arxiv.org/abs/1406.2338), followed by [minimum-weight perfect matching](https://arxiv.org/abs/2303.15933), on the toric code with side length $L$. We consider i.i.d. $X$ errors on the qubits (bonds), and work with $m$ anyons (plaquettes). The errors are plotted on the dual lattice by default.
## 2D Decoder
At each time step out of $T=L$ total, perform the following steps:
1. Make an $X$ error on each qubit with probability $p=0.05$.
2. Perform $c$ field updates, according to the following rule:$$\phi\mapsto\phi+\frac\eta4\nabla^2\phi+q$$Where $\phi$ is the auxillary field, $\eta=0.1$ is the Jacobi smoothing parameter, and $q$ is the anyon density with unit mass $-\frac4\eta$. Take $4\pi G=1$, so that $\nabla^2\phi=\rho$.
3. For each anyon, move it its highest-$\phi$ neighboring cell with probability $\frac12$.

## Notebook Initialization

In [5]:
%matplotlib widget

In [6]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
mako = sns.color_palette("mako", as_cmap=True)

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150
plt.ioff()

from tqdm.notebook import tqdm
import os
from multiprocess import Pool
try:
    num_cpus = len(os.sched_getaffinity(0))
except:
    num_cpus = os.cpu_count()
print("Using {} CPUs".format(num_cpus))

import sys
sys.path.insert(0, 'toric-decoder')
from toric import *
from pymatching import Matching

Using 4 CPUs


## Example

In [7]:
# Simulation parameters

L = 100 # Lattice size
p_error = 0.05 # Error probability per spin
η = 0.1 # Smoothing paramter for Jacobi method
c = 16 # "Field velocity" - number of field updates per cycle
T = L # Epochs

matching = Matching(pcm(L))

In [44]:
mystate = init_state(L, p_error)
initial_correction = mwpm(matching, mystate.q)
print("Fail after MWPM: {}".format(logical_error(initial_correction ^ mystate.error)))

q_history, error_history = decoder_2D(mystate, T, c, η, p_error = 0, history = True)
print("Fail after 2D: {}".format(mystate.N > 0 or logical_error(mystate.error)))

correction = mwpm(matching, q_history[-1,:,:])
q_history = np.append(q_history, [np.zeros_like(mystate.q)], axis = 0)
error_history = np.append(error_history, [mystate.error ^ correction], axis = 0)
print("Fail after 2D and MWPM: {}".format(logical_error(error_history[-1,:,:,:])))

Fail after MWPM: True
Fail after 2D: True
Fail after 2D and MWPM: False


In [45]:
plot_evolution(q_history, error_history, dual = True)

## MC Simulation
### Parameter Space
* System size $L\in\left\{20,40,\dotsc,200\right\}$
* Field velocity $c\in\left\{2,4,\dotsc,64\right\}$
* Initial error rate: $p=0.05$
* Error rate per $\tau$ time steps: $p=0.05$, $\tau=1$
* Time steps $T=L$
* Smoothing factor $\eta=0.1$
* Shots: $10^4$
### Recorded Data
* Failure rate 2D + MWPM
* Density of anyons $N/L^2$

In [8]:
# Simulation parameters

L = 100 # Lattice size
p_error = 0.05 # Error probability per spin
η = 0.1 # Smoothing paramter for Jacobi method
c = 16 # "Field velocity" - number of field updates per cycle
T = L # Epochs

matching = Matching(pcm(L))

In [9]:
def f(n):
    mystate = init_state(L, p_error)
    initial_correction = mwpm(matching, mystate.q)
    mwpm_fail = logical_error(initial_correction ^ mystate.error)

    decoder_2D(mystate, T, c, η, p_error = 0, history = False)
    ca_fail = (mystate.N > 0 or logical_error(mystate.error))

    correction = mwpm(matching, mystate.q)
    ca_mwpm_fail = logical_error(correction ^ mystate.error)
    return mwpm_fail, ca_fail, ca_mwpm_fail

In [10]:
%%time

with Pool(num_cpus) as p:
    result = p.map(f, range(100))

mwpm_fail, ca_fail, ca_mwpm_fail = np.sum(result, axis = 0)
print("MWPM fail rate: {}".format(mwpm_fail))
print("CA fail rate: {}".format(ca_fail))
print("CA+MWPM fail rate: {}".format(ca_mwpm_fail))

MWPM fail rate: 0
CA fail rate: 92
CA+MWPM fail rate: 0
CPU times: user 28.5 ms, sys: 30.4 ms, total: 58.9 ms
Wall time: 9.15 s
